In [61]:
import geopandas as gpd
from datetime import datetime, date, timedelta
import numpy as np
from salishsea_tools import geo_tools

### open shapefile and explore a bit

In [5]:
shapedir  = '/Users/rmueller/Data/MIDOSS/cargo_2018_01/'
shapefile = 'cargo_2018_01.shp'

In [6]:
data = gpd.read_file(shapedir + shapefile)

In [70]:
[nrows,ncols]=data.shape
nrows

296982

In [10]:
for c in data.columns:
    print(c)


MMSI
ST_DATE
EN_DATE
LENGTH
TO
FROM_
geometry


In [14]:
data.geometry[0].coords[:]

[(-125.91133833331847, 48.00089666672749),
 (-125.90970166717838, 48.00166833288793)]

In [17]:
data.LENGTH[100000]

198

### determine vessel time exposure as time / distance for poly line segment

#### start with time

In [30]:
data.EN_DATE[0] 

'2018-01-01 05:29:33'

In [35]:
start_date = datetime.strptime(data.ST_DATE[0] , '%Y-%m-%d %H:%M:%S')
end_date = datetime.strptime(data.EN_DATE[0] , '%Y-%m-%d %H:%M:%S')

In [45]:
delta_time = end_date - start_date
delta_time.total_seconds()

22.0

#### now distance

In [54]:
# two coordination locations in each line segment of geometry
line = 0
first = data.geometry[line].coords[0]
second = data.geometry[line].coords[1]

In [50]:
type(first)

tuple

In [60]:
lat = np.zeros(2)
lon = np.zeros(2)
lon[0] = first[0]
lat[0] = first[1]
lon[1] = second[0]
lat[1] = second[1]

In [64]:
# calculate the distance in km of vessel line segment
geo_tools.haversine(lon[0],lat[0],lon[1],lat[1])

0.14887232475160814

#### Calculate distance from a given lat/lon location to polyline

In [65]:
# use a test lat/lon that came from get_lat_lon_indices
spill_lon = -124.9243
spill_lat = 50.18442
search_radius = 0.5 # km

##### Dip a toe in first

In [ ]:
# use the first lat/lon location to identify line segments within 500 m radiu
lon = [data.geometry[i].coords[0][0] for i in range(3)]
lat = [data.geometry[i].coords[0][1] for i in range(3)]

In [105]:
# apply the haversine function to calculate distance from spill location
# to poly line segments (based on first location of poly line) 
distance = geo_tools.haversine(spill_lon, spill_lat, lon, lat)

In [106]:
distance

array([ 253.04542028,  252.92927167,  252.87606897])

In [108]:
array_index = np.where(distance < 260)
array_index

(array([0, 1, 2]),)

In [109]:
array_index = np.where(distance < 10)
array_index

(array([], dtype=int64),)

##### Water feels good.  Dive in!

In [110]:
display('this is going to take a minute....')

'this is going to take a minute....'

In [111]:
lon_array = [data.geometry[i].coords[0][0] for i in range(nrows)]
lat_array = [data.geometry[i].coords[0][1] for i in range(nrows)]

In [116]:
distance = geo_tools.haversine(spill_lon, spill_lat, lon_array, lat_array)
array_index = np.where(distance < search_radius)
array_index

(array([56654]),)

In [142]:
total_seconds  = np.zeros(len(array_index))
total_distance = np.zeros(len(array_index))
vte            = np.zeros(len(array_index))

# loop through values in array_index values to calculate 
for index in range(len(array_index)):
    
    # calculate the duration of travel for each poly line segment
    start_date = datetime.strptime(data.ST_DATE[index] , '%Y-%m-%d %H:%M:%S')
    end_date = datetime.strptime(data.EN_DATE[index] , '%Y-%m-%d %H:%M:%S')
    delta_time = end_date - start_date
    total_seconds[index] = delta_time.total_seconds()
    
    # calculate the distance of each poly line
    start_lon = data.geometry[index].coords[0][0]
    start_lat = data.geometry[index].coords[0][1]
    end_lon   = data.geometry[index].coords[1][0]
    end_lat   = data.geometry[index].coords[1][1]
    
    # calculate the distance in km of vessel line segment
    total_distance[index] = geo_tools.haversine( start_lon, start_lat, end_lon, end_lat )
    vte[index] = total_seconds[index]/total_distance[index]
    
# find the index for greatest vte (for cases where there is more than one polyline)    
i, = np.where(vte == max(vte))
the_one = array_index[i.item()]


### now that we have found the one true polyline, get its digits!
we want vessel length, to and from

In [148]:
length = data.LENGTH[the_one.item()]
origin = data.FROM_[the_one.item()]
destination = data.TO[the_one.item()]

In [150]:
print(length)
print(origin)
print(destination)

16
None
None
